In [55]:
from V6_tratamiento import tratamiento, tratamiento2
from utils import *

import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE

# Modelos Selección.
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Modelos.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold

# Métricas 
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, classification_report

In [56]:
df_modelado = pd.read_pickle("./pickles_temp/V7.1.pkl") 
df_modelado.head(1)

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2,status_group
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,...,False,False,False,True,False,False,False,719,788,2


In [57]:
df_modelado.shape

(59400, 66)

In [58]:
# Rebalanceo

print("Before OverSampling, counts of label '2': {}".format(sum(df_modelado.status_group == 2)))
print("Before OverSampling, counts of label '1': {}".format(sum(df_modelado.status_group == 1)))
print("Before OverSampling, counts of label '0': {}".format(sum(df_modelado.status_group == 0)))

# Definir la instancia SMOTE
sm = SMOTE(random_state=42)

# Aplicar SMOTE
df_modelado_resampled, y_resampled = sm.fit_resample(df_modelado.drop('status_group', axis=1), df_modelado.status_group.ravel())

# Imprimir la forma del DataFrame resultante después de aplicar SMOTE
print('After OverSampling, the shape of train_X: {}'.format(df_modelado_resampled.shape))
print('After OverSampling, the shape of train_y: {}'.format(y_resampled.shape))

# Imprimir los recuentos de cada etiqueta en la columna 'status_group' después de aplicar SMOTE
print("After OverSampling, counts of label '2': {}".format(sum(y_resampled == 2)))
print("After OverSampling, counts of label '1': {}".format(sum(y_resampled == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled == 0)))


Before OverSampling, counts of label '2': 32259
Before OverSampling, counts of label '1': 4317
Before OverSampling, counts of label '0': 22824
After OverSampling, the shape of train_X: (96777, 65)
After OverSampling, the shape of train_y: (96777,)
After OverSampling, counts of label '2': 32259
After OverSampling, counts of label '1': 32259
After OverSampling, counts of label '0': 32259


In [59]:
df_modelado_resampled

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
0,69572,6000.000000,1,3376,275,1390.000000,3477,94,34.938093,-9.856322,...,False,False,False,False,True,False,False,False,719,788
1,8776,0.000000,0,1709,53,1399.000000,1750,39,34.698766,-2.147466,...,False,True,False,False,False,False,False,False,384,966
2,34310,25.000000,1,2413,8,686.000000,4397,408,37.460664,-3.821329,...,False,False,False,False,False,True,False,False,770,718
3,67743,0.000000,0,3962,1057,263.000000,4002,222,38.486161,-11.155298,...,False,False,False,False,False,True,False,False,571,16
4,19728,0.000000,0,794,1,1527.000000,906,135,31.130847,-1.825359,...,False,True,False,False,False,False,False,False,591,472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96772,69487,0.000000,0,2396,1240,1506.843764,1386,17402,31.734041,-1.947274,...,False,False,False,False,True,True,False,False,548,610
96773,59649,0.000000,0,2476,537,1245.950301,1655,693,31.152607,-6.433508,...,False,False,False,True,True,False,True,False,268,458
96774,40506,852.810277,1,4188,881,716.485249,4331,192,37.806223,-4.109765,...,False,False,False,False,True,False,False,False,673,302
96775,2782,0.000000,0,1693,9084,1276.601846,1386,17402,30.873197,-2.831729,...,False,False,False,True,True,False,False,False,347,561


In [60]:
y_resampled.shape # Y es una serie o df con la variable objetivo

(96777,)

In [61]:
# Sigo la guia de churn_operadora
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split (df_modelado_resampled, 
                                                     y_resampled,
                                                     test_size = 0.2,
                                                     random_state = 0,
                                                     stratify = y_resampled)

In [62]:
# 2'
classifier5 = RandomForestClassifier(criterion = 'entropy', n_estimators= 500, max_depth = 16).fit(X_train_res, y_train_res)#quito max_features
pred5 = classifier5.predict(X_test_res)

In [63]:
print(classification_report(y_test_res,pred5))
# Parece muy buen resultado

              precision    recall  f1-score   support

           0       0.88      0.71      0.78      6452
           1       0.87      0.90      0.88      6452
           2       0.77      0.89      0.83      6452

    accuracy                           0.83     19356
   macro avg       0.84      0.83      0.83     19356
weighted avg       0.84      0.83      0.83     19356



In [64]:
# Funcion no apta para modelos multiclase
def saca_metricas(y1, y2):
    saca_metricas(y_test,y_pred)
    print('matriz de confusión')
    print(confusion_matrix(y1, y2))
    print('accuracy')
    print(accuracy_score(y1, y2))
    print('precision')
    print(precision_score(y1, y2))
    print('recall')
    print(recall_score(y1, y2))
    print('f1')
    print(f1_score(y1, y2))
    false_positive_rate, recall, thresholds = roc_curve(y1, y2)
    roc_auc = auc(false_positive_rate, recall)
    print('AUC')
    print(roc_auc)
    plt.plot(false_positive_rate, recall, 'b')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.title('AUC = %.2f' % roc_auc) 
if 0:
    saca_metricas(y_test_res,pred5)

In [65]:
# Lo entreno de nuevo para subirlo. Para no complicarme lo entreno con unn split al 0.99
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split (df_modelado_resampled, 
                                                     y_resampled,
                                                     test_size = 0.01,
                                                     random_state = 0,
                                                     stratify = y_resampled)

In [66]:
# docu: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
classifier6 = RandomForestClassifier(criterion = 'entropy', n_estimators= 500, max_depth = 16).fit(X_train_res, y_train_res)#quito max_features
pred6 = classifier6.predict(X_test_res)

In [74]:
# Quiero ver por curiosidad que hace exactamente
X_test_res

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
96340,36996,217.58737,0,1178,734,1356.113489,1623,404,37.292509,-3.453983,...,False,False,True,False,True,False,False,False,662,795
79010,6577,0.00000,0,3573,721,679.322037,1386,17402,34.259745,-6.877552,...,False,False,False,True,True,False,False,False,341,803
68497,68437,0.00000,0,2999,20,1224.556223,3055,19,32.228719,-2.081373,...,False,False,False,True,True,False,False,False,332,547
62797,20924,0.00000,0,2628,590,1268.036118,2049,222,34.898594,-5.529890,...,False,False,False,False,False,True,False,False,267,301
2021,10299,0.00000,0,4234,1246,1400.000000,4397,408,32.125763,-3.719510,...,False,False,False,True,False,False,False,False,715,727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36608,63195,0.00000,0,4234,1246,399.500000,1247,116,32.987709,-4.128995,...,False,False,False,True,False,False,False,False,176,872
55239,63136,500.00000,1,3965,11,1724.000000,1386,17402,34.700531,-9.107210,...,False,False,True,False,False,False,False,False,876,578
50294,25421,200.00000,1,2634,87,1672.000000,2789,87,35.467055,-7.941660,...,False,False,False,False,True,False,False,False,241,295
26752,2432,0.00000,0,3423,270,1245.500000,1386,17402,31.944902,-3.590110,...,False,False,False,True,False,False,False,False,375,309


In [67]:
pred6

array([1, 1, 1, 0, 2, 0, 2, 1, 1, 0, 2, 2, 2, 1, 1, 2, 2, 2, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 2, 2, 2, 2, 1, 1, 1, 0, 2, 0, 2, 0, 2, 2, 1, 1, 1,
       0, 1, 2, 0, 1, 2, 1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 2, 1, 2, 1, 0,
       1, 1, 1, 1, 0, 0, 2, 2, 2, 1, 1, 2, 1, 1, 0, 2, 1, 0, 0, 2, 2, 1,
       1, 2, 0, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1,
       0, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 2, 2, 1, 1, 2, 2, 0, 1, 1,
       2, 0, 2, 2, 2, 1, 1, 2, 1, 1, 2, 0, 1, 2, 0, 1, 1, 1, 2, 2, 1, 0,
       0, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 0, 2, 2, 1, 0, 2,
       0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 1, 2, 1, 1, 1, 0, 1, 1, 2, 0,
       0, 2, 2, 2, 2, 1, 2, 0, 1, 2, 1, 0, 1, 2, 1, 2, 0, 0, 0, 2, 2, 2,
       1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 0, 2, 1, 1, 0,
       2, 2, 0, 2, 2, 1, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 0,
       0, 1, 1, 0, 2, 0, 1, 2, 1, 1, 0, 1, 2, 0, 0,

In [68]:
print(classification_report(y_test_res,pred6))

              precision    recall  f1-score   support

           0       0.84      0.67      0.75       323
           1       0.89      0.92      0.90       323
           2       0.75      0.88      0.81       322

    accuracy                           0.82       968
   macro avg       0.83      0.82      0.82       968
weighted avg       0.83      0.82      0.82       968



In [69]:
df_target = pd.read_csv('./test set values.csv', sep=',')
df_target = tratamiento(df_target)
df_target = tratamiento2(df_target)
df_target.head(1)

Este df no tiene status_group


,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,...,source_type_spring,source_class_groundwater,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
0,50785,0.0,174,1996.0,213,35.290799,-4.059696,867,0,3952,...,False,False,True,False,False,False,False,True,789,60


In [70]:
def tratamiento3(df):
    columnas_buenas = ['id', 'amount_tsh', 'amount_tsh_bool', 'funder',
       'funder_frequency_encoded', 'gps_height', 'installer',
       'installer_frequency_encoded', 'longitude', 'latitude', 'wpt_name',
       'wpt_name_frequency_encoded', 'num_private_b', 'basin', 'subvillage',
       'subvillage_frequency_encoded', 'region', 'region_frequency_encoded',
       'region_code', 'district_code', 'lga', 'lga_frequency_encoded', 'ward',
       'ward_frequency_encoded', 'population', 'public_meeting', 'permit',
       'construction_year', 'year_recorded', 'management_Parastatal',
       'management_Private operator', 'management_VWC', 'management_WUA',
       'management_WUG', 'management_Water Board',
       'management_Water authority', 'extraction_type_class_motorpump',
       'extraction_type_class_other', 'extraction_type_class_rope pump',
       'extraction_type_class_submersible',
       'extraction_type_class_wind-powered', 'management_group_other/unknown',
       'management_group_user-group', 'payment_type_monthly',
       'payment_type_on failure', 'payment_type_other/unknown',
       'payment_type_per bucket', 'quality_group_fluoride',
       'quality_group_milky', 'quality_group_salty', 'quality_group_unknown',
       'quantity_enough', 'quantity_insufficient', 'quantity_seasonal',
       'source_type_dam', 'source_type_other',
       'source_type_rainwater harvesting', 'source_type_river/lake',
       'source_type_shallow well', 'source_type_spring',
       'waterpoint_type_communal standpipe multiple',
       'waterpoint_type_improved spring', 'waterpoint_type_other', 'random1',
       'random2']
    df = df[columnas_buenas]
    return(df)

df_target = tratamiento3(df_target)
df_target.shape

(14850, 65)

In [71]:
df_target

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
0,50785,0.0,0,174,174,1996.0,213,36,35.290799,-4.059696,...,False,True,False,False,False,False,False,True,789,60
1,51630,0.0,0,249,249,1569.0,220,4690,36.656709,-3.309214,...,False,False,False,False,True,False,False,False,220,291
2,17168,0.0,0,814,814,1567.0,220,4690,34.767863,-5.004344,...,False,True,False,False,False,False,False,True,190,866
3,45559,0.0,0,221,221,267.0,261,8,38.058046,-9.418672,...,False,False,False,True,False,False,False,True,922,227
4,49871,500.0,1,72,72,1260.0,74,9,35.006123,-10.950412,...,False,False,False,False,True,False,False,False,985,620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,0,144,144,34.0,225,64,38.852669,-6.582841,...,False,False,True,False,False,False,False,False,843,182
14846,18990,1000.0,1,285,285,816.5,318,3,37.451633,-5.350428,...,False,False,False,True,False,False,False,False,624,854
14847,28749,0.0,0,249,249,1476.0,220,4690,34.739804,-4.585587,...,False,False,False,False,False,False,False,False,487,141
14848,33492,0.0,0,241,241,998.0,220,4690,35.432732,-10.584159,...,False,False,True,False,False,False,False,False,981,148


In [78]:
df_prediccion_subir = classifier6.predict(df_target) # clasifier6 es el que corrige el desbalanceo
df_prediccion_subir.shape
# Solo tiene una dimension. Tengo que correlacionarlo con el ID para subirlo.... Voy a hacer un concat

(14850,)

In [80]:
df_prediccion_subir = pd.DataFrame(df_prediccion_subir, columns=['prediccion_subir'])

df_target = pd.concat([df_target, df_prediccion_subir], axis=1)
df_target.head(1)

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2,prediccion_subir
0,50785,0.0,0,174,174,1996.0,213,36,35.290799,-4.059696,...,True,False,False,False,False,False,True,789,60,0
1,51630,0.0,0,249,249,1569.0,220,4690,36.656709,-3.309214,...,False,False,False,True,False,False,False,220,291,2
2,17168,0.0,0,814,814,1567.0,220,4690,34.767863,-5.004344,...,True,False,False,False,False,False,True,190,866,0
3,45559,0.0,0,221,221,267.0,261,8,38.058046,-9.418672,...,False,False,True,False,False,False,True,922,227,0
4,49871,500.0,1,72,72,1260.0,74,9,35.006123,-10.950412,...,False,False,False,True,False,False,False,985,620,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,0,144,144,34.0,225,64,38.852669,-6.582841,...,False,True,False,False,False,False,False,843,182,0
14846,18990,1000.0,1,285,285,816.5,318,3,37.451633,-5.350428,...,False,False,True,False,False,False,False,624,854,2
14847,28749,0.0,0,249,249,1476.0,220,4690,34.739804,-4.585587,...,False,False,False,False,False,False,False,487,141,2
14848,33492,0.0,0,241,241,998.0,220,4690,35.432732,-10.584159,...,False,True,False,False,False,False,False,981,148,2


In [81]:
df_target = df_target[['id', 'prediccion_subir']]

In [83]:
df_target.head(1)

,id,prediccion_subir
0,50785,0


In [85]:
df_target = df_target.rename(columns={'prediccion_subir': 'status_group'})
df_target

,id,status_group
0,50785,0
1,51630,2
2,17168,0
3,45559,0
4,49871,2
...,...,...
14845,39307,0
14846,18990,2
14847,28749,2
14848,33492,2


In [87]:
df_target.set_index('id', inplace=True)
df_target['status_group'].replace([0, 1, 2], ['non functional', 'functional needs repair', 'functional'], inplace=True)
df_target
#Vamos a probar en la web

,status_group
id,
50785,non functional
51630,functional
17168,non functional
45559,non functional
49871,functional
...,...
39307,non functional
18990,functional
28749,functional


In [88]:
df_target.to_pickle("./pickles_temp/V7.3.pkl")
df_target.to_csv("./resultados/V7.3.csv")
# El resultado de la web es 0.7416
# Parece que cada vez que toco algo va a peor